In [73]:
import numpy as np
import pandas as pd
import mygene

### Prep-processing raw TF + gene file

In [ ]:
# preprocessing transcription factor + gene regulation list
# filename = "../data/TF_network/pathway_extensions_v2.transcriptional.listt"
# cols = ["TF"] + ["gene"+str(i) for i in range(2500)]
# df = pd.read_csv(filename, names=cols, delimiter="\t", low_memory=False)
# valid_cols = df.columns[~df.isnull().all()]
# df = df[valid_cols]
# df.replace(np.nan, 0).to_csv("../data/TF_network/TF_gene.csv", index=None)

### convert TF+gene file into a matrix of 0s and 1s

In [ ]:
df = pd.read_csv("../data/TF_network/TF_gene.csv", low_memory=False)
pathway_genes = sorted(list(set(df.as_matrix().flatten())))
pathway_genes.remove('0')
assert(len(df["TF"]) == len(set(df["TF"])))
assert(len(pathway_genes) == len(set(pathway_genes)))
TFs = sorted(list(df["TF"]))

df_mask = pd.DataFrame(index=TFs, columns=pathway_genes)
for row_idx, TF in enumerate(TFs):
    if row_idx % 500 == 0:
        print(row_idx)
    TF_genes = set(df[df.TF == TF].as_matrix().flatten())
    TF_genes.remove('0')
    Not_TF_genes = [i for i in pathway_genes if i not in TF_genes]
    df_mask.loc[TF, TF_genes] = 1 
    df_mask.loc[TF, Not_TF_genes] = 0
df_mask.to_csv("../data/TF_network/TF_gene_matrix.csv")
df_mask.to_hdf("../data/TF_network/TF_gene_matrix.hdf", "TF")

### filter for genes that exist in TCGA

In [75]:
tcga_genes = list(pd.read_csv("../data/gene_ids.txt", header=None)[0])
tcga_genes = [i.split("|")[0] for i in tcga_genes]
df_mask = pd.read_hdf("../data/TF_network/TF_gene_matrix.hdf")
pathway_genes = list(df_mask.columns)
common_genes = sorted(list(set(tcga_genes).intersection(pathway_genes)))

print("number of genes in tcga", len(set(tcga_genes)))
print("number of genes in pathway", len(pathway_genes))
print("number of common genes between tcga and pathway", len(common_genes))

df_mask_tcga = df_mask[common_genes]
common_tfs = [i for i in df_mask.index if i in common_genes]
df_mask_tcga = df_mask_tcga.loc[common_tfs]
print("shape of TF_gene_matrix after filtering for genes existing in TCGA:")
print(df_mask_tcga.shape)
df_mask_t

df_mask_tcga.to_hdf("../data/TF_network/TF_gene_matrix_filtered_by_tcga.hdf", "TF")
df_mask_tcga.to_csv("../data/TF_network/TF_gene_matrix_filtered_by_tcga.csv")

number of genes in tcga 16331
number of genes in pathway 14237
number of common genes between tcga and pathway 11104
